In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold

# Load the URL dataset
url_data = pd.read_csv('url/url.csv')

# Check the data
print(url_data.head())

# Load metadata
url_metadata = pd.read_csv('url/url_metadata.csv')

# Inspect metadata (for feature info)
print(url_metadata.head())


   length_url  length_hostname  ip  nb_dots  nb_hyphens  nb_at  nb_qm  nb_and  \
0          37               19   0        3           0      0      0       0   
1          77               23   1        1           0      0      0       0   
2         126               50   1        4           1      0      1       2   
3          18               11   0        2           0      0      0       0   
4          55               15   0        2           2      0      0       0   

   nb_or  nb_eq  ...  suspecious_tld  statistical_report  \
0      0      0  ...               0                   0   
1      0      0  ...               0                   0   
2      0      3  ...               0                   0   
3      0      0  ...               0                   0   
4      0      0  ...               0                   0   

   whois_registered_domain  domain_registration_length  domain_age  \
0                        0                          45          -1   
1           

In [18]:
# Extract features (X) and target (y)
X = url_data.drop(columns=['is_phishing'])  # Features (exclude target)
y = url_data['is_phishing']  # Target variable (is phishing)

# Handle missing values by imputing with the mean for numerical columns
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Further validation if needed (cross-validation for robust training)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train a classifier (you can replace this with any other model)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model on the test set
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")


Model Accuracy: 0.9343797025371829


In [23]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from joblib import Parallel, delayed

def clip(x, lower_bound, upper_bound):
    """
    Clips the adversarial example to make sure it stays within the allowed bounds.
    """
    return np.clip(x, lower_bound, upper_bound)

def generate_adversarial_sample(model, X_sample, y_sample, metadata, epsilon=1, maxiters=20, alpha=0.01, lambda_=0.1, weights=None):
    """
    Generates an adversarial sample for a given data point using the LowProFool attack.
    This function is parallelizable.
    """
    r = np.zeros_like(X_sample)
    orig_pred = model.predict(X_sample)
    loop_i = 0
    best_pert_x = X_sample.copy()
    best_norm_weighted = np.inf
    
    while loop_i < maxiters:
        # Compute the model output for the perturbed sample
        perturbed_sample = X_sample + r
        output_pred = model.predict(perturbed_sample)

        # Set target label (assuming binary classification with flipped labels)
        target = 1 - orig_pred  # Flip the target class

        # Compute the loss: cross-entropy loss + L2 regularization term (simplified for this example)
        loss_1 = 0 if output_pred == target else 1  # Misclassification penalty (simplified)
        loss_2 = np.linalg.norm(r * weights)  # L2 norm penalty
        loss = loss_1 + lambda_ * loss_2

        # Compute gradient of the loss with respect to the perturbation (simplified)
        grad_r = np.sign(r) * loss_2  # Placeholder gradient (in practice, you would compute this numerically)

        # Apply perturbation in the direction of the negative gradient
        ri = -grad_r * alpha  # Scale the gradient by alpha

        # Update the perturbation
        r += ri

        # Clip perturbation to ensure it's within the feature bounds
        r = clip(r, 0, epsilon)

        # Update the adversarial example
        xprime = X_sample + r

        # Check if adversarial example fools the model and has a lower norm
        if output_pred != orig_pred and np.sum(np.abs(r * weights)) < best_norm_weighted:
            best_norm_weighted = np.sum(np.abs(r * weights))
            best_pert_x = xprime

        loop_i += 1

    # Final clipping to ensure example stays within bounds
    best_pert_x = clip(best_pert_x, 0, epsilon)
    
    return best_pert_x

def lowprofool_attack(model, X, y, metadata, epsilon=1, maxiters=20, alpha=0.01, lambda_=0.1, batch_size=32, n_jobs=-1):
    """
    This function generates adversarial examples using the LowProFool approach in a parallelized manner.
    """
    # Feature weights (importance) initialized from metadata or a default value
    weights = np.ones(X.shape[1])
    orig_pred = model.predict(X)

    # Use joblib to parallelize the adversarial sample generation
    adversarial_samples = Parallel(n_jobs=n_jobs)(
        delayed(generate_adversarial_sample)(model, X[i:i+1], y[i:i+1], metadata, epsilon, maxiters, alpha, lambda_, weights)
        for i in range(X.shape[0])
    )

    # Convert the result into a numpy array
    adversarial_samples = np.vstack(adversarial_samples)

    # Evaluate model on adversarial examples
    y_adv_pred = model.predict(adversarial_samples)
    print(f"Model Accuracy on Adversarial Data: {accuracy_score(y, y_adv_pred)}")

    return adversarial_samples, y_adv_pred


# Example usage:
# Assuming `model` is a trained RandomForestClassifier and `X_train`, `y_train`, and `url_metadata` are available

for epoch in range(10):  # Set the number of epochs as needed
    print(f"Epoch {epoch + 1} started.")
    
    # Train on clean data
    model.fit(X_train, y_train)
    
    # Generate adversarial data using batch processing and parallelization
    X_adv, _ = lowprofool_attack(model, X_train, y_train, url_metadata, batch_size=64, n_jobs=-1)
    
    # Train on adversarial data (this can be done on the same model for simplicity)
    model.fit(X_adv, y_train)
    
    # Optionally evaluate model after each epoch
    y_train_pred = model.predict(X_train)
    print(f"Epoch {epoch + 1} - Accuracy on Clean Data: {accuracy_score(y_train, y_train_pred)}")
    
    # Evaluate on adversarial data as well
    y_train_pred_adv = model.predict(X_adv)
    print(f"Epoch {epoch + 1} - Accuracy on Adversarial Data: {accuracy_score(y_train, y_train_pred_adv)}")


Epoch 1 started.
Model Accuracy on Adversarial Data: 0.7713254593175853
Epoch 1 - Accuracy on Clean Data: 0.9386876640419947
Epoch 1 - Accuracy on Adversarial Data: 0.9386876640419947
Epoch 2 started.
Model Accuracy on Adversarial Data: 0.7713254593175853
Epoch 2 - Accuracy on Clean Data: 0.9386876640419947
Epoch 2 - Accuracy on Adversarial Data: 0.9386876640419947
Epoch 3 started.
Model Accuracy on Adversarial Data: 0.7713254593175853
Epoch 3 - Accuracy on Clean Data: 0.9386876640419947
Epoch 3 - Accuracy on Adversarial Data: 0.9386876640419947
Epoch 4 started.
Model Accuracy on Adversarial Data: 0.7713254593175853
Epoch 4 - Accuracy on Clean Data: 0.9386876640419947
Epoch 4 - Accuracy on Adversarial Data: 0.9386876640419947
Epoch 5 started.
Model Accuracy on Adversarial Data: 0.7713254593175853
Epoch 5 - Accuracy on Clean Data: 0.9386876640419947
Epoch 5 - Accuracy on Adversarial Data: 0.9386876640419947
Epoch 6 started.
Model Accuracy on Adversarial Data: 0.7713254593175853
Epoch 6 

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from joblib import Parallel, delayed

def clip(x, lower_bounds, upper_bounds):
    """
    Clips adversarial examples to feature-wise bounds.
    """
    return np.minimum(np.maximum(x, lower_bounds), upper_bounds)

def generate_adversarial_sample(model, X_sample, y_sample, metadata, epsilon=1, maxiters=20, alpha=0.01, lambda_=0.1, weights=None):
    """
    Generates an adversarial sample for a given data point.
    """
    r = np.zeros_like(X_sample)
    orig_pred = model.predict(X_sample)
    best_pert_x = X_sample.copy()
    best_norm_weighted = np.inf
    loop_i = 0

    # Get feature bounds from metadata
    lower_bounds = metadata["lower_bounds"]
    upper_bounds = metadata["upper_bounds"]

    while loop_i < maxiters:
        perturbed_sample = X_sample + r
        output_pred = model.predict(perturbed_sample)
        target = (orig_pred + 1) % model.n_classes_  # Multi-class example

        loss_1 = 0 if output_pred == target else 1
        loss_2 = np.linalg.norm(r * weights)
        loss = loss_1 + lambda_ * loss_2

        grad_r = np.sign(r) * alpha  # Simplified; replace with proper computation if possible
        r -= grad_r
        r = clip(r, -epsilon, epsilon)

        xprime = X_sample + r
        xprime = clip(xprime, lower_bounds, upper_bounds)

        if output_pred != orig_pred and np.sum(np.abs(r * weights)) < best_norm_weighted:
            best_norm_weighted = np.sum(np.abs(r * weights))
            best_pert_x = xprime

        loop_i += 1

    return best_pert_x

def lowprofool_attack(model, X, y, metadata, epsilon=1, maxiters=20, alpha=0.01, lambda_=0.1, batch_size=32, n_jobs=-1):
    """
    Generates adversarial examples using LowProFool in parallel.
    """
    weights = metadata.get("weights", np.ones(X.shape[1]))
    lower_bounds = metadata["lower_bounds"]
    upper_bounds = metadata["upper_bounds"]

    adversarial_samples = Parallel(n_jobs=n_jobs)(
        delayed(generate_adversarial_sample)(
            model, X[i:i+1], y[i:i+1], metadata, epsilon, maxiters, alpha, lambda_, weights
        )
        for i in range(X.shape[0])
    )

    adversarial_samples = np.vstack(adversarial_samples)
    y_adv_pred = model.predict(adversarial_samples)
    print(f"Model Accuracy on Adversarial Data: {accuracy_score(y, y_adv_pred)}")

    return adversarial_samples, y_adv_pred
